In [1]:
import numpy as np
import csv
import os 
import sys
import pdb
import string

## Chris's Rhyming Scorer

In [2]:
# imports
import re
from pathlib import Path
from typing import List, Union

import numpy as np
import pronouncing

from youshen.util import calculate_edit_distance, clean

LIMERICK_PATTERN = [[0, 1], [2, 3], [0, 4]]
BLACKLIST =[
    "=+.*=+",
    "(<.endoftext)*.>"
]


def get_top_limerick(text: str):
    """
    Parameters
    ----------
    text: Str
        Samples of text generated by GPT-2

    Returns
    ----------
    top_limerick: str
        The best rhyming limerick from a set of samples generated by GPT-2

    Examples
    ----------
    >>> from pathlib import Path
    >>> from youshen.poem import get_top_limerick
    >>> samples = Path("samples/samples.txt")
    >>> with open(samples) as file:
    >>>     text = file.read()
    >>> top = get_top_limerick(text)
    """
    poem_samples = text.split("<|endoftext|>")

    poems = [
        clean(poem_sample, BLACKLIST) 
        for poem_sample in poem_samples 
        if len(poem_sample) > 0
    ]

    poems = [poem for poem in poems if len(poem) > 0]
    grouped_verses = [[line for line in poem.splitlines() if line] for poem in poems ]
    poem_text = ["\n".join(grouped_verse) for grouped_verse in grouped_verses if len(grouped_verse)>4] 
    poems = [SamplePoem(text=poem_sample, rhyme_patterns=LIMERICK_PATTERN, verse_length=5) 
                 for poem_sample in poem_text if len(poem_sample) >4]
    indexed_scores = {poems.index(poem): poem.get_rhyme_score() for poem in poems}
    top_poem_index = max(indexed_scores)
    top_limerick = poems[top_poem_index].lines
    return "\n".join(top_limerick)

class Limerick:
    def __init__(self, lines: str, rhyme_patterns: List, max_length: int = None):
        self.verse_lines = lines
        if max_length:
            self.verse_lines = self.verse_lines[0:max_length]
#         pdb.set_trace()
        try:
            self.last_words = [line.split()[-1] for line in self.verse_lines] # Error here
        except:
            pdb.set_trace()
        self.last_word_rhyming_part_pairs = {
            word: self.__get_rhyming_parts(word) for word in self.last_words
        }
        self.rhyme_patterns = rhyme_patterns

    def __get_phonemes(self, text: Union[str, List]):
        """returns all possible pronunciation of a word as phonemes
        Language used: American English. Style: Arpabet
        """
        if type(text) == str:
            phonemes = pronouncing.phones_for_word(text)
        else:
            phonemes = [pronouncing.phones_for_word(word) for word in text]
        return phonemes

    def __get_rhyming_parts(self, word: str):
        phonemes = self.__get_phonemes(word)
        rhyming_parts = [pronouncing.rhyming_part(phoneme) for phoneme in phonemes]
        return rhyming_parts

    def __get_valid_rhyme_patterns(self):
        valid_patterns = [
            pattern
            for pattern in self.rhyme_patterns
            if not any(i > len(self.verse_lines) - 1 for i in pattern)
        ]
        return valid_patterns

    def score(self, line_pair: List):
        first_word = self.last_words[line_pair[0]]
        second_word = self.last_words[line_pair[1]]
        first_word_rhymes = self.__get_rhyming_parts(first_word)
        second_word_rhymes = self.__get_rhyming_parts(second_word)
        rhyme_score = 0
        for first_word_rhyme in first_word_rhymes:
            for second_word_rhyme in second_word_rhymes:
                is_rhyming = first_word_rhyme == second_word_rhyme
                if is_rhyming:
                    rhyme_score = 1
                    status = "successfully matched"
                else:
                    status = "could not match"
                # uncomment to debug
                print(
                    f" {status} -> {first_word}({first_word_rhyme}) and {second_word}({second_word_rhyme})"
                )
        return int(rhyme_score)

    def get_rhyme_score(self):
        """returns a rhyming score for the poem between 0 and 1.
        """
        valid_patterns = self.__get_valid_rhyme_patterns()
        scores = [self.score(pattern) for pattern in valid_patterns]
        if len(scores) == 0:
            return 0
            # pdb.set_trace()
        return sum(scores) / len(scores)

    def __repr__(self):
        return repr("\n".join(self.verse_lines))


class SamplePoem:
    def __init__(
        self, text: str, rhyme_patterns: List, verse_length: int, blacklist: List = None
    ):
#         pdb.set_trace()
        self.lines = [line for line in text.splitlines() if line]
        self.verse_length = verse_length
        self.rhyme_patterns = rhyme_patterns
        intervals = list(range(0, len(self.lines), verse_length))
        verse_lines_list = [self.lines[x : x + 5] for x in intervals]
        self.verses = [
            Limerick(
                lines=verse_lines,
                rhyme_patterns=self.rhyme_patterns,
                max_length=self.verse_length,
            )
            for verse_lines in verse_lines_list
        ]

    def __get_item__(self, key):
        return self.verses[key]

    def get_rhyme_score(self):
        if len(self.verses):
            scores = [verse.get_rhyme_score() for verse in self.verses]
            score = sum(scores) / len(scores)
        else:
            score = None
        return score

    def __repr__(self):
        return repr(self.verses)


def read_poems(file_path: Path, blacklist):
    """reads a file containing poems and returns a list of limerick samples found in the file
    """
    with open(file_path) as file:
        text = file.read()
    poem_samples = text.split("<|endoftext|>")
    poems = [clean(sample, blacklist) for sample in poem_samples if len(sample) > 0]
    return [poem for poem in poems if len(poem) > 0]


def score_poems(file_path: Path, last_word_pattern: str, blacklist: List):
    """Reads limericks in generated samples and scores them between 0 and 1
    """
    poems = [
        SamplePoem(text=poem_sample, rhyme_patterns=LIMERICK_PATTERN, verse_length=5)
        for poem_sample in read_poems(file_path, blacklist=blacklist)
        if len(poem_sample) > 0
    ]
    poems = [poem for poem in poems if poem.lines]
    poem_scores = [poem.get_rhyme_score() for poem in poems]
    return poem_scores


def test_scoring_limerick(sample_rhyme: Path, limerick_pattern: List):
    """Sanity check to test scoring of a single limerick
    """
    with open(sample_rhyme) as rhyme_sample:
        sample_corpus = rhyme_sample.read()
    limerick_lines = [line for line in sample_corpus.splitlines() if line]
    limerick = Limerick(
        lines=limerick_lines, rhyme_patterns=limerick_pattern, max_length=5
    )
    print("Scoring limerick...")
    score = limerick.get_rhyme_score()
    print(f"Rhyme score is {score}")
    assert type(score) == float


## Sperate Text and Score

In [3]:
filename = "samples_singh.txt"
limericks = []
limerick = []
with open(filename, "r") as file:
    
    for x in file:
        
        if "======================================== " in x:
            continue 
        elif "<|endoftext|>" in x :
            if len(limerick) == 5:
                limericks.append(limerick)
            limerick = []
            continue
        elif x == '\n' or x == ' \n':
            continue
        
#         x = x.replace('\n', '')
        limerick.append(x)
            
poems = [SamplePoem(text=''.join(poem_sample), rhyme_patterns=LIMERICK_PATTERN, verse_length=5) 
         for poem_sample in limericks if len(poem_sample) >4]
indexed_scores = {poems.index(poem): poem.get_rhyme_score() for poem in poems} 

 could not match -> dispensed(EH1 N S T) and abused(UW1 Z D)
 could not match -> intentions(EH1 N CH AH0 N Z) and detractors(AE1 K T ER0 Z)
 could not match -> intentions(EH1 N CH AH0 N Z) and detractors(AE1 K T ER0 Z)
 successfully matched -> sod(AA1 D) and god(AA1 D)
 could not match -> who(UW1) and from(AH1 M)
 could not match -> substance(AH1 B S T AH0 N S) and vision(IH1 ZH AH0 N)
 successfully matched -> who(UW1) and screw(UW1)
 successfully matched -> guy(AY1) and eye(AY1)
 could not match -> arouses(AW1 Z IH0 Z) and engineering(IH1 R IH0 NG)
 successfully matched -> guy(AY1) and high(AY1)
 successfully matched -> no(OW1) and below(OW1)
 successfully matched -> no(OW1) and below(OW1)
 successfully matched -> no(OW1) and go(OW1)
 successfully matched -> led(EH1 D) and bed(EH1 D)
 could not match -> status(AE1 T AH0 S) and said(EH1 D)
 could not match -> status(EY1 T AH0 S) and said(EH1 D)
 successfully matched -> led(EH1 D) and head(EH1 D)
 could not match -> agent(EY1 JH AH0 N T

 could not match -> smitten(IH1 T AH0 N) and tumbled(AH1 M B AH0 L D)
 could not match -> tearful(IH1 R F AH0 L) and bad(AE1 D)
 could not match -> smitten(IH1 T AH0 N) and whittled(IH1 T AH0 L D)
 could not match -> smitten(IH1 T AH0 N) and whittled(IH1 T AH0 L D)
 successfully matched -> bride(AY1 D) and hide(AY1 D)
 successfully matched -> bed(EH1 D) and said(EH1 D)
 successfully matched -> bride(AY1 D) and denied(AY1 D)
 successfully matched -> pay(EY1) and play(EY1)
 could not match -> daughter(AO1 T ER0) and jonathan(AA1 N AH0 TH AH0 N)
 could not match -> she(IY1) and a(AH0)
 could not match -> she(IY1) and a(EY1)
 could not match -> daughter(AO1 T ER0) and horrid(AO1 R AH0 D)
 could not match -> fought(AO1 T) and caught(AA1 T)
 successfully matched -> fought(AO1 T) and caught(AO1 T)
 successfully matched -> who(UW1) and too(UW1)
 successfully matched -> fought(AO1 T) and ought(AO1 T)
 could not match -> stress(EH1 S) and x(EH1 K S)
 successfully matched -> explain(EY1 N) and co

 successfully matched -> spell(EH1 L) and tell(EH1 L)
 successfully matched -> right(AY1 T) and sight(AY1 T)
 could not match -> obama(AA1 M AH0) and demeanors(IY1 N ER0 Z)
 successfully matched -> worth(ER1 TH) and mirth(ER1 TH)
 could not match -> obama(AA1 M AH0) and ala(AE1 L AH0)
 could not match -> obama(AA1 M AH0) and ala(EY1 L AH0)
 could not match -> peter(IY1 T ER0) and her(ER1)
 could not match -> peter(IY1 T ER0) and her(HH ER0)
 could not match -> pole(OW1 L) and soft(AA1 F T)
 could not match -> pole(OW1 L) and soft(AO1 F T)
 successfully matched -> peter(IY1 T ER0) and peter(IY1 T ER0)
 successfully matched -> air(EH1 R) and dare(EH1 R)
 successfully matched -> ears(IH1 R Z) and rears(IH1 R Z)
 could not match -> ears(IY1 R Z) and rears(IH1 R Z)
 successfully matched -> air(EH1 R) and faire(EH1 R)
 successfully matched -> other(AH1 DH ER0) and mother(AH1 DH ER0)
 successfully matched -> old(OW1 L D) and hold(OW1 L D)
 successfully matched -> other(AH1 DH ER0) and other(A

 successfully matched -> dread(EH1 D) and said(EH1 D)
 successfully matched -> mind(AY1 N D) and mind(AY1 N D)
 successfully matched -> mind(AY1 N D) and find(AY1 N D)
 successfully matched -> fish(IH1 SH) and wish(IH1 SH)
 successfully matched -> fish(IH1 SH) and wish(IH1 SH)
 successfully matched -> ed(EH1 D) and lead(EH1 D)
 could not match -> ed(EH1 D) and lead(IY1 D)
 successfully matched -> appear(IH1 R) and clear(IH1 R)
 could not match -> ed(EH1 D) and seed(IY1 D)
 could not match -> petting(EH1 T IH0 NG) and dressing(EH1 S IH0 NG)
 could not match -> fool(UW1 L) and employ(OY1)
 could not match -> fool(UW1 L) and employ(OY1)
 successfully matched -> think(IH1 NG K) and drink(IH1 NG K)
 successfully matched -> bad(AE1 D) and mad(AE1 D)
 successfully matched -> think(IH1 NG K) and pink(IH1 NG K)
 could not match -> admonished(AA1 N IH0 SH T) and trembled(EH1 M B AH0 L D)
 successfully matched -> under(AH1 N D ER0) and wonder(AH1 N D ER0)
 could not match -> admonished(AA1 N IH0 

 successfully matched -> fred(EH1 D) and bed(EH1 D)
 could not match -> bay(EY1) and parry(EH1 R IY0)
 successfully matched -> fred(EH1 D) and fred(EH1 D)
 could not match -> gnawing(AO1 IH0 NG) and churning(ER1 N IH0 NG)
 successfully matched -> rage(EY1 JH) and age(EY1 JH)
 successfully matched -> shore(AO1 R) and shore(AO1 R)
 successfully matched -> food(UW1 D) and brood(UW1 D)
 successfully matched -> shore(AO1 R) and adore(AO1 R)
 successfully matched -> fit(IH1 T) and split(IH1 T)
 successfully matched -> brake(EY1 K) and break(EY1 K)
 successfully matched -> fit(IH1 T) and split(IH1 T)
 could not match -> chamomile(AY1 L) and you(UW1)
 could not match -> chamomile(IY1 L) and you(UW1)
 could not match -> also(AO1 L S OW0) and vowel(AW1 AH0 L)
 could not match -> the(DH AH0) and places(EY1 S AH0 Z)
 could not match -> the(DH AH0) and places(EY1 S IH0 Z)
 could not match -> the(AH1) and places(EY1 S AH0 Z)
 could not match -> the(AH1) and places(EY1 S IH0 Z)
 could not match -> th

...

 successfully matched -> fare(EH1 R) and air(EH1 R)
 successfully matched -> fare(EH1 R) and there(EH1 R)
 successfully matched -> request(EH1 S T) and chest(EH1 S T)
 successfully matched -> request(EH1 S T) and chest(EH1 S T)
 successfully matched -> quench(EH1 N CH) and bench(EH1 N CH)
 successfully matched -> quench(EH1 N CH) and stench(EH1 N CH)
 successfully matched -> round(AW1 N D) and bound(AW1 N D)
 could not match -> thought(AO1 T) and caught(AA1 T)
 successfully matched -> thought(AO1 T) and caught(AO1 T)
 successfully matched -> round(AW1 N D) and sound(AW1 N D)
 successfully matched -> hello(OW1) and dough(OW1)
 successfully matched -> hello(OW1) and dough(OW1)
 could not match -> jaw(AO1) and raw(AA1)
 could not match -> hello(OW1) and flew(UW1)
 could not match -> hello(OW1) and flew(UW1)
 could not match -> reno(IY1 N OW0) and sue(UW1)
 could not match -> pox(AA1 K S) and rich(IH1 CH)
 could not match -> reno(IY1 N OW0) and screw(UW1)
 could not match -> twos(UW1 Z) an

 could not match -> horse(AO1 R S) and late(EY1 T)
 successfully matched -> delay(EY1) and convey(EY1)
 could not match -> meaning(IY1 N IH0 NG) and definition(IH1 SH AH0 N)
 could not match -> addition(IH1 SH AH0 N) and meaning(IY1 N IH0 NG)
 could not match -> meaning(IY1 N IH0 NG) and definition(IH1 SH AH0 N)
 successfully matched -> fight(AY1 T) and right(AY1 T)
 successfully matched -> sure(UH1 R) and pure(UH1 R)
 could not match -> breakfasts(EH1 K F AH0 S T S) and tempt(EH1 M P T)
 could not match -> it(IH1 T) and kinder(AY1 N D ER0)
 could not match -> it(IH0 T) and kinder(AY1 N D ER0)
 could not match -> duller(AH1 L ER0) and much(AH1 CH)
 successfully matched -> it(IH1 T) and it(IH1 T)
 could not match -> it(IH1 T) and it(IH0 T)
 could not match -> it(IH0 T) and it(IH1 T)
 successfully matched -> it(IH0 T) and it(IH0 T)
 could not match -> obese(IY1 S) and curse(ER1 S)
 could not match -> obese(IY1 S) and curse(ER1 S)
 could not match -> obese(IY1 S) and worse(ER1 S)
 could n

 could not match -> called(AO1 L D) and charge(AA1 R JH)
 could not match -> stage(EY1 JH) and male(EY1 L)
 successfully matched -> gale(EY1 L) and snail(EY1 L)
 successfully matched -> cave(EY1 V) and knave(EY1 V)
 successfully matched -> gale(EY1 L) and bale(EY1 L)
 could not match -> uk(EY1) and in(IH0 N)
 could not match -> uk(EY1) and in(IH1 N)
 could not match -> whips(IH1 P S) and beast(IY1 S T)
 could not match -> whips(IH1 P S) and beast(IY1 S T)
 successfully matched -> dad(AE1 D) and pad(AE1 D)
 successfully matched -> dad(AE1 D) and pad(AE1 D)
 successfully matched -> thrives(AY1 V Z) and thrives(AY1 V Z)
 could not match -> dump(AH1 M P) and bug(AH1 G)
 could not match -> thrives(AY1 V Z) and lives(IH1 V Z)
 successfully matched -> thrives(AY1 V Z) and lives(AY1 V Z)
 successfully matched -> shape(EY1 P) and ape(EY1 P)
 successfully matched -> sharks(AA1 R K S) and sharks(AA1 R K S)
 could not match -> shape(EY1 P) and drift(IH1 F T)
 could not match -> bucket(AH1 K AH0 T)

 successfully matched -> frown(AW1 N) and crown(AW1 N)
 could not match -> handle(AE1 N D AH0 L) and swinger(IH1 NG ER0)
 successfully matched -> secure(UH1 R) and sure(UH1 R)
 successfully matched -> high(AY1) and dry(AY1)
 could not match -> secure(UH1 R) and mess(EH1 S)
 successfully matched -> me(IY1) and tree(IY1)
 could not match -> remain(EY1 N) and retains(EY1 N Z)
 could not match -> remain(EY1 N) and retains(EY1 N Z)
 could not match -> remain(EY1 N) and retains(EY1 N Z)
 could not match -> remain(EY1 N) and retains(EY1 N Z)
 successfully matched -> me(IY1) and me(IY1)
 could not match -> near(IH1 R) and bizarre(AA1 R)
 could not match -> near(IH1 R) and bizarre(AA1 R)
 could not match -> behaviors(EY1 V Y ER0 Z) and envelopes(OW2 P S)
 could not match -> near(IH1 R) and archival(AY1 V AH0 L)
 successfully matched -> root(UW1 T) and shoot(UW1 T)
 could not match -> edge(EH1 JH) and dance(AE1 N S)
 successfully matched -> root(UW1 T) and fruit(UW1 T)
 successfully matched -> z

 could not match -> eat(IY1 T) and seeking(IY1 K IH0 NG)
 could not match -> is(IH1 Z) and driving(AY1 V IH0 NG)
 could not match -> is(IH0 Z) and driving(AY1 V IH0 NG)
 successfully matched -> eat(IY1 T) and wheat(IY1 T)
 successfully matched -> eat(IY1 T) and wheat(IY1 T)
 could not match -> named(EY1 M D) and saris(AA1 R IY0 Z)
 could not match -> named(EY1 M D) and is(IH1 Z)
 could not match -> named(EY1 M D) and is(IH0 Z)
 successfully matched -> son(AH1 N) and none(AH1 N)
 could not match -> seldom(EH1 L D AH0 M) and smarter(AA1 R T ER0)
 successfully matched -> son(AH1 N) and one(AH1 N)
 could not match -> motel(EH1 L) and holt(OW1 L T)
 successfully matched -> brave(EY1 V) and slave(EY1 V)
 successfully matched -> pie(AY1) and lie(AY1)
 could not match -> stuff(AH1 F) and pull(UH1 L)
 could not match -> pie(AY1) and rise(AY1 Z)
 successfully matched -> say(EY1) and way(EY1)
 successfully matched -> books(UH1 K S) and looks(UH1 K S)
 could not match -> rage(EY1 JH) and hour(AW1 

In [4]:
len(limericks)

8371

In [5]:
inds = [k for k,v in indexed_scores.items() if v == 1.0]

In [6]:
len(inds)

1097

In [ ]:
out = [limericks[i] for i in inds]

In [ ]:
filename = "best_limericks.txt"
with open(filename, "w") as file:
    for lim in out:
        file.write(''.join(lim)+'\r\n')
#         pdb.set_trace()